# Tarea 3: implementar un modelo de inferencia sin servidor

## Tarea 3.1: configuración del entorno

Instale los paquetes y las dependencias.

In [ ]:
#install-dependencies
import boto3
import sagemaker
import sagemaker_datawrangler
import sys
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

Guarde el modelo del laboratorio de entrenamiento y ajuste en el bucket predeterminado de Amazon Simple Storage Service (Amazon S3). Configure un modelo usando **create_model** y configure **ModelDataUrl** para referenciar al modelo entrenado.

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(
    Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz"
)

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

## Tarea 3.2: crear un punto de enlace a partir del modelo reentrenado y sintetizado que se proporciona

Inferencia sin servidor de Amazon SageMaker es una opción de inferencia personalizada que le permite implementar y escalar modelos de machine learning (ML). Inferencia sin servidor es ideal para las cargas de trabajo que tienen períodos de inactividad entre los aumentos de tráfico y pueden tolerar el arranque en frío. Los puntos de enlace sin servidor inician automáticamente recursos de cómputo y los escalan y reducen horizontalmente según el tráfico. Por lo tanto, no necesita seleccionar tipos de instancias ni administrar políticas de escalado. Esto elimina el trabajo pesado no diferenciado de seleccionar y administrar servidores. La inferencia sin servidor se integra en AWS Lambda para ofrecerle una disponibilidad alta, la tolerancia a errores integrada y el escalado automático.

Puede crear un punto de enlace sin servidor con el SDK para Python de Amazon SageMaker en tres pasos. Los pasos son los mismos que se usan para el punto de enlace en tiempo real, pero tienen distintas configuraciones:
1. Cree un modelo en SageMaker.
2. Cree una configuración para un punto de enlace HTTPS.
3. Cree un punto de enlace HTTPS.

Ya creó un modelo. Ahora está listo para crear un punto de enlace y su configuración. 

Primero, establezca el nombre de la configuración del punto de enlace y el tamaño de la memoria que desea usar. Luego, llame a la API CreateEndpointConfig.

Para crear una configuración del punto de enlace, debe establecer las siguientes opciones:
- **VariantName**: el nombre de la variante de producción (uno o más modelos en producción)
- **ModelName**: el nombre del modelo que desea alojar. Este es el nombre que especificó cuando creó el modelo.
- **ServerlessConfig**: aquí es donde se configura el punto de enlace como “sin servidor”. Configure los valores de **MemorySizeInMB** y **MaxConcurrency**.
    - **MemorySizeInMB**: el tamaño de memoria asignado (1024, 2048, 3072, 4096, 5120 o 6144 MB).
    - **MaxConcurrency**: la cantidad de invocaciones simultáneas (1 a 200).

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date so you can search endpoints based on creation time.
endpoint_config_name = 'income-model-serverless-endpoint-{}'.format(create_date)                              

endpoint_config_response = sm.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "variant1", # The name of the production variant
            "ServerlessConfig": {
                "MemorySizeInMB": 2048, # The memory size
                "MaxConcurrency": 20 # Number of concurrent invocations
            }
        } 
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

A continuación, cree un punto de enlace. Cuando crea un punto de enlace sin servidor, SageMaker aprovisiona y administra los recursos de procesamiento por usted. Luego, usted puede hacer solicitudes de inferencias al punto de enlace y, como respuesta, recibir predicciones sobre el modelo. SageMaker escala y reduce los recursos de cómputo horizontalmente según sea necesario para gestionar el tráfico de su solicitud, y usted solo paga por lo que usa.

Puede elegir entre un contenedor proporcionado por SageMaker o uno propio. Un punto de enlace sin servidor tiene un tamaño de RAM mínimo de 1024 MB y un tamaño de RAM máximo de 6144 MB. La inferencia sin servidor asigna automáticamente recursos de cómputo proporcionales a la memoria que selecciona.

Cuando el punto de enlace está en funcionamiento, la función auxiliar imprime el Amazon Resource Name (ARN) del punto de enlace. La creación del punto de enlace puede demorar hasta 7 minutos en ejecutarse.

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


En SageMaker Studio, puede ver los detalles del punto de enlace debajo de la pestaña **Endpoints** (Puntos de enlace).

El próximo paso abre una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una vista de pantalla dividida de la ventana principal de SageMaker Studio, arrastre la pestaña **serverless_inference.ipynb** hacia el lado o seleccione (con el botón derecho del mouse) la pestaña **serverless_inference.ipynb** y elija **New View for Notebook** (Nueva vista para el cuaderno). Ahora puede tener las instrucciones visibles mientras explora el punto de enlace.
- **Opción 2**: alternar entre las pestañas de SageMaker Studio para seguir estas instrucciones. Cuando termine de explorar el punto de enlace, seleccione la pestaña **serverless_inference.ipynb** para volver al cuaderno.

1. Seleccione el ícono de **SageMaker Home** (Inicio de SageMaker).
2. Seleccione **Deployments** (Implementaciones).
3. Seleccione **Endpoints** (Puntos de enlace).

Verá la pestaña **Endpoints** (Puntos de enlace) en SageMaker Studio.

4. Seleccione el punto de enlace que tenga **income-model-serverless-** en la columna **Name** (Nombre).

Si el punto de enlace no aparece, seleccione el ícono de actualización hasta que aparezca.

Verá la pestaña **ENDPOINT DETAILS** (DETALLES DEL PUNTO DE ENLACE) en SageMaker Studio.

5. Vaya a la pestaña **AWS settings** (Configuración de AWS).

Si abrió el punto de enlace antes de que terminara de crearse, seleccione el ícono de actualización hasta que el valor de **Endpoint status** (Estado del punto de enlace) cambie de *Creating* (Creando) a *InService* (En funcionamiento).

En **Endpoint type** (Tipo de punto de enlace), verá **Serverless** (Sin servidor). En la sección **Endpoint runtime settings** (Configuración del tiempo de ejecución del punto de enlace), verá las configuraciones que eligió antes en el cuaderno.

## Tarea 3.3: invocar un punto de enlace para una inferencia sin servidor con registros de clientes

Después de que implemente su modelo con los servicios de alojamiento de SageMaker, puede enviarle datos de prueba al modelo en ese punto de enlace para probarlo.

Si su punto de enlace no recibe tráfico por un tiempo y, luego, comienza a recibir nuevas solicitudes de repente, es posible que necesite un poco de tiempo para poner en marcha los recursos de cómputo y procesarlas. Esto se denomina arranque en frío. Dado que los puntos de enlace sin servidor aprovisionan recursos de cómputo bajo demanda, su punto de enlace podría experimentar arranques en frío. El arranque en frío también puede producirse si sus solicitudes simultáneas superan el uso actual de solicitudes simultáneas. El tiempo del arranque en frío depende del tamaño de su modelo y del tiempo que demora su descarga, así como del tiempo de inicio de su contenedor.

Consulte [Inferencia sin servidor](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html) para obtener más información sobre el funcionamiento de la inferencia sin servidor y el arranque en frío.

Recibió varios registros más de clientes. Confirme que el punto de enlace esté funcionando. Para ello, invóquelo con un conjunto de registros que tengan un valor de ingresos de 1 y un valor de ingresos de 0. La salida será una lista de las puntuaciones de las predicciones para cada registro. 

In [ ]:
#invoke-endpoint-serverless-records
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('47,0,4,9,0,3,4,0,1,0,1902,60\n' +
                '53,0,0,0,0,2,4,0,1,0,0,40\n' +
                '44,0,0,0,2,0,1,0,1,14344,0,40\n', 'utf-8')
)

print(response)

print('\nTesting with records that have an income value of 1:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

start_time = time.time()
response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('19,0,1,1,1,1,2,1,0,0,0,35\n' +
                '56,2,1,1,0,1,0,0,0,0,0,50\n' +
                '61,2,0,0,0,0,0,0,0,0,0,40\n', 'utf-8')
)

print('\nTesting with records that have an income value of 0:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

## Tarea 3.4: eliminar el punto de enlace

La eliminación de un punto de enlace se puede lograr en tres pasos. Primero, elimine el punto de enlace. Luego, elimine la configuración del punto de enlace. Por último, si ya no necesita el modelo que implementó, elimínelo.

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### Conclusión

¡Felicitaciones! Usó SageMaker para crear correctamente un punto de enlace sin servidor, con el SDK para Python de SageMaker, y para invocarlo.

La próxima tarea del laboratorio se centra en la implementación de un modelo de inferencia usando una inferencia asíncrona.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Tarea 4: implementar un modelo de inferencia asíncrona**.